In [1]:
from PIL import Image
import numpy as np
import pandas as pd
import os
import sys
import glob
import time
import itertools
import random
import scipy.misc
import pickle  

from PyQt5 import QtWidgets
import matplotlib
matplotlib.use('Qt5Agg')   # 允许GUI式matplotlib，即有弹窗. it must be set before import pyplot
import matplotlib.pyplot as plt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5 import NavigationToolbar2QT as NavigationToolbar
from matplotlib.figure import Figure


In [2]:
# path
class path:  
    def __init__(self, abs_path_str=''):
        ""
        self.abs_path_str = abs_path_str
#         , i_dir=1, i_subdir=1, i_img=1
#         self.i_dir = i_dir
#         self.i_subdir = i_subdir
#         self.i_img = i_img
        self.i_dir = 0
        self.i_subdir = 0
        self.i_img = 0
        if self.abs_path_str == '':
            self.abs_path_str = input("请输入数据文件夹的绝对路径：如 E:\zhouhan")
        assert os.path.exists(self.abs_path_str), "invalid dir path"  # 必须填入有效的起始文件夹data的绝对路径名称
        
        self.update_path()
        self.regulate_dir()
        self.update_path()
    
    def update_path(self):
        # update dirs
        items = os.listdir(self.abs_path_str)  # PATH下的所有item
        self.dirs = [item for item in items if os.path.isdir(os.path.join(self.abs_path_str, item))] # 所有item中的所有文件夹
        self.n_dirs = len(self.dirs)
#         self.dirs = dict(zip(range(1,len(self.dirs)+1,1),self.dirs))
        # 如果没有创建 子文件夹 的话，按照.hsv文件的名称创建，并以列表的字典形式{dir: list_subdir}记录所有subdirs
        self.subdirs = {}
        for i, dir in enumerate(self.dirs):
            list_hsv = glob.glob(self.abs_path_str+'\\'+dir+'\\*.hsv') # 列出该文件夹下所有hsv文件的路径
            list_subdir = [s.split('.')[0].split("\\")[-1] for s in list_hsv] # 列出该文件夹下所有hsv文件的名字，注意，不包括后缀.hsv
            self.subdirs[dir] = list_subdir  # 更新subdirs词典
#             dct_subdir = dict(zip(range(1,len(list_subdir)+1,1),list_subdir))
#             self.subdirs[dir] = dct_subdir  # 更新subdirs词典

            # 如果没有创建子文件夹，那么逐文件夹新建和.hsv文件同名的子文件夹
            for subdir in list_subdir:
                if not os.path.exists(self.abs_path_str+"\\"+dir+"\\"+subdir):
                    os.mkdir(self.abs_path_str+"\\"+dir+"\\"+subdir) 
        self.n_subdirs = {}
        for i, dir in enumerate(self.dirs):
            self.n_subdirs[dir] = len(self.subdirs[dir])
        self.n_img = 3767
        print("update path done!")
        return None
        
    def regulate_dir(self):
        pass
        # 改成全大写
        # 改名，将所有experiments级文件夹名由小写改为大写并重新获取目录
        for dir in self.dirs:
            if dir.islower():
                os.rename(self.abs_path_str+"\\"+dir, self.abs_path_str+"\\"+dir.upper())        
        # 去掉空格
        print("regulate_dir done!")
        self.update_path()
        return None
    
    def display(self, n_dir=None):
        print("------list of directories------")
        for i,j in enumerate(self.dirs, start=1):
            print(str(i).rjust(2) + ":" + str(j).rjust(12))    
        if n_dir:
            print("------list of subdirectories------")
            for i,j in enumerate(self.subdirs[self.dirs[n_dir-1]], start=1):
                print(str(i).rjust(2) + ":" + str(j).rjust(12))
        return None
    
    def return_path(self):
#         assert(- self.n_dirs <= self.i_dir < self.n_dirs and
#                - self.n_subdirs[self.dirs[self.i_dir]] <= self.i_subdir < self.n_subdirs[self.dirs[self.i_dir]] and
#                - self.n_img <= self.i_img <= self.n_img), "invalid index of dir or subdir"

        self.dir_name = self.dirs[self.i_dir]
        self.subdir_name = self.subdirs[self.dirs[self.i_dir]][self.i_subdir]
        self.img_name = self.subdir_name + '_' + str(self.i_img).zfill(4) + '.jpg'
        return self.abs_path_str+"\\"+self.dir_name+"\\"+self.subdir_name + "\\" +self.img_name
    
    def set_path(self,i_dir, i_subdir, i_img):
        self.i_dir = i_dir
        self.i_subdir = i_subdir
        self.i_img = i_img
        if self.i_dir > 0:
            self.i_dir -= 1
        if self.i_subdir > 0:
            self.i_subdir -= 1
#         if self.i_img > 0:
#             self.i_img -= 1
        return None

    def get_path(self,i_dir, i_subdir, i_img):
        if i_dir > 0:
            i_dir -= 1
        if i_subdir > 0:
            i_subdir -= 1
#         if i_img > 0:
#             i_img -= 1
        assert (- self.n_dirs <= i_dir < self.n_dirs and
                - self.n_subdirs[self.dirs[i_dir]] <= i_subdir < self.n_subdirs[self.dirs[i_dir]] and
                - self.n_img <= i_img < self.n_img), "invalid index of dir or subdir"

        dir_name = self.dirs[i_dir]
        subdir_name = self.subdirs[self.dirs[i_dir]][i_subdir]
        img_name = subdir_name + '_' + str(i_img).zfill(4) + '.jpg'
        return self.abs_path_str+"\\"+dir_name+"\\"+subdir_name + "\\" +img_name        

    def return_range(self, i_dir, i_subdir, i_img, stop=3767, step=1):
        self.set_path(i_dir, i_subdir, i_img)
        assert (- self.n_dirs <= i_dir < self.n_dirs and
                - self.n_subdirs[self.dirs[i_dir]] <= i_subdir < self.n_subdirs[self.dirs[i_dir]] and
                - self.n_img <= i_img < self.n_img and
                1 <= stop <= self.n_img)
        for i in range(i_img,stop+1,1):
            if not (- self.n_dirs <= self.i_dir < self.n_dirs and
                    - self.n_subdirs[self.dirs[self.i_dir]] <= self.i_subdir < self.n_subdirs[self.dirs[self.i_dir]] and
                    - self.n_img <= self.i_img < min(stop, self.n_img)+1):
                raise StopIteration
            else:
                yield self.return_path()                  
                self.i_img += step
        
#     def __next__(self, step=1):
#         if not (- self.n_dirs <= self.i_dir < self.n_dirs and
#                 - self.n_subdirs[self.dirs[self.i_dir]] <= self.i_subdir < self.n_subdirs[self.dirs[self.i_dir]] and
#                 - self.n_img <= self.i_img < self.n_img):
#             raise StopIteration
#         else:
#             self.i_img += step
#             return self.return_path()      

In [3]:
paths = path("E:\zhouhan")

update path done!
regulate_dir done!
update path done!
update path done!


In [4]:
##paths.display()

In [5]:
# paths.display(n_dir=2)

In [6]:
# paths.get_path(2,1,1)

In [7]:
class breakage_distribution:
    def __init__(self,paths=paths):
        HEADER = ['x', 'y', 'circle', 'dir', 'pic', 'subdir', 'timestamp']
        self.pts = pd.DataFrame(columns=HEADER)
        self.lens = pd.DataFrame(columns=[])
    
    def get_pts(self, i_dir, i_subdir, i_img, stop=3767, step=1):
        """循环读取图片，然后标注，结束循环后输入n"""
        pts = []
        for p in paths.return_range(i_dir=i_dir, i_subdir=i_subdir, i_img=i_img, stop=stop, step=step):
            im = Image.open(p)
            plt.imshow(im)
            title_pic = p.split(".")[0].split(" / ")[-1].split("_")[-1]
            start = int(title_pic)
            print("正在标注%s:" % title_pic)            
            plt.title(title_pic)
            grid_X,grid_Y =  zip(*[(200,100),(200,600), (200,100),(1000,100),
                                   (200,350),(1000,350),(400,100),(400,600),
                                   (600,100),(600,600), (800,100),(800,600)])
            for n_grid in range(min(len(grid_X),len(grid_Y)) // 2):
                plt.plot(grid_X[2*n_grid: 2*n_grid+2], grid_Y[2*n_grid: 2*n_grid+2],'k--',lw=1)
            grid_X,grid_Y =  zip(*[(1000,100),(1000,600),(200,600),(1000,600)])
            for n_grid in range(min(len(grid_X),len(grid_Y)) // 2):
                plt.plot(grid_X[2*n_grid: 2*n_grid+2], grid_Y[2*n_grid: 2*n_grid+2],'k-',lw=1)
            tmp_pts = []
            tmp_pts = plt.ginput(n=-1, timeout=0, show_clicks=True)
            # 如果数据不合格就重新标注一下当前
            if not self.isvalid(tmp_pts):
                print("重新标注%s" % title_pic)
                tmp_pts = []
                self.get_pts(i_dir, i_subdir, i_img=start, stop=start+1)
                continue
            pts += tmp_pts
            print("完成小标注%s:" % title_pic)  
            # 
        reply = input("""Again! press "a"
                             Quit! press "q"
                             Done! press "d" """)
#             plt.show()               
        # 整理格式为DataFrame 
        print("完成标注 %s:" %title_pic)
        
        return self.pts
    
    def pts2len(self):
        def get_d_equiv(df):
            """输入连续的4条坐标信息，返回这四个点确定的圆的直径"""
        #     print("\n\n------------------\n")
        #     print(df)
            x = df['x'].values
            y = df['y'].values
        #     print(x,"\n")
        #     print(y,"\n")
        #     x = [3,0,3,6]
        #     y = [4,0,8,4]
            laxis = np.hypot(abs(x[0]-x[1]), abs(y[0]-y[1]))  # 长轴
            saxis = np.hypot(abs(x[2]-x[3]), abs(y[2]-y[3]))  # 短轴
            get_geo_mean = lambda x: np.sqrt(x[0]*x[1])
        #     print(get_geo_mean((laxis, saxis)))
            return get_geo_mean((laxis, saxis))


        data_diameters = data_copy.groupby(by=["dir", "subdir", 'pic', "circle"])[['x','y']].apply(get_d_equiv)
        # data_diameters
        data_diameters.to_csv(PREFIX+'_diameters.csv', header=["diameter"])

        data_diameters = data_diameters.reset_index(level=[0,1,2,3])

        data_diameters.rename_axis({0:"diameter"},axis=1)
        data_diameters.to_csv(PREFIX+"_diameters.csv", header=True)

        data_diameters.head(10)

    def isvalid(self,pts):
        # 如果pts是空的
#         if len(pts) == 0:
#             return False
        # 如果不满足 正好4个一组
        if np.mod(len(pts), 4) != 0:
            return False
        # 如果每组四个点两两之间的距离过大（大于250）
        for start, stop in zip(range(0, len(pts)-1, 4), range(4, len(pts)+1, 4)): 
            for a, b in itertools.combinations(pts[start: stop], 2): 
                if (abs(a[0] - b[0]) > 250 or abs(a[1] - b[1]) > 250):
                    return False
        if False:# 如果没有按照对角线顺序选取
            return False
    #     if s=input("Sure to sumbit?") # 用户确认功能
        else:
            return True


In [8]:
bd = breakage_distribution(paths)

In [ ]:
bd.get_pts(2,1,1,stop=3767, step=500)

正在标注0001:


C:\ProgramData\Anaconda3\envs\stat\lib\site-packages\matplotlib\backend_bases.py:2453: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


完成小标注0001:
正在标注0002:
完成小标注0002:
正在标注0003:
重新标注0003
正在标注0003:
完成小标注0003:
正在标注0004:


In [ ]:
print("""save the result above? input "s" 
""")

In [ ]:
def 
for p in paths.return_range(i_dir=2, i_subdir=1, i_img=1, stop=11, step=5):
    
    for img_index, img_path in zip(range(iimg,stop,step), img_paths):
        name_experiment = img_path.split(".")[0].split("\\")[-1].split("_")[0]
        name_pic = img_path.split(".")[0].split(" / ")[-1].split("_")[-1]
        print("正在标注%s:" % name_pic)
        im = Image.open(img_path)
        plt.imshow(im)
        grid_X,grid_Y =  zip(*[(200,100),(200,600), (200,100),(1000,100),
                               (200,350),(1000,350),(400,100),(400,600),
                               (600,100),(600,600), (800,100),(800,600)])
        for n_grid in range(min(len(grid_X),len(grid_Y)) // 2):
            plt.plot(grid_X[2*n_grid: 2*n_grid+2], grid_Y[2*n_grid: 2*n_grid+2],'k--',lw=1)
        grid_X,grid_Y =  zip(*[(1000,100),(1000,600),(200,600),(1000,600)])
        for n_grid in range(min(len(grid_X),len(grid_Y)) // 2):
            plt.plot(grid_X[2*n_grid: 2*n_grid+2], grid_Y[2*n_grid: 2*n_grid+2],'k-',lw=1)
        
        plt.title(name_experiment + ": " + str(idir) + " / " + str(isubdir) + " / " + name_pic)

        pts = []
#         plt.text(10, 10, "Notes: must pick points in diagonal order, or will not pass the test and redo")
        pts = plt.ginput(n=-1, timeout=0, show_clicks=True)

        
        # 如果数据不合格就重新标注一遍
        if not isvalid(pts):
            print("重新标注%s" % name_pic)
            pts = []
            load_image_path(idir, isubdir, iimg=img_index, stop=img_index+1)
            continue
        
        print("完成标注 %s:" %name_pic)
        
        # 保存数据
        n_circle = int(len(pts)/4)
        circle = []
        for i in range(n_circle):
            circle += [i] * 4
        df_circle = pd.DataFrame(circle, columns=['circle'])
        df_dir = pd.DataFrame({'dir':[dirs[idir]]*len(pts),
                               'subdir': [subdirs[dirs[idir]][isubdir]]*len(pts), 
                               "timestamp":[time.asctime()]*len(pts),
                               "pic":name_pic})
        df_pts = pd.DataFrame(pts, columns=list("xy"), copy=True)
        df_data = pd.concat([df_pts, df_circle,df_dir], axis=1)
        print(df_data)
        # if os.getcwd()+
        df_data.to_csv(os.getcwd()+"\\data_" + str(subdirs[dirs[idir]][isubdir]) + ".csv",mode='a', header=False)
        print("数据保存完毕")
    print("所有数据保存完毕，程序退出")
    plt.close()
    return None

In [ ]:

# ********************************
# DO NOT CHANGE ANYTHING BELOW !!!
# ********************************


In [172]:
paths.set_path(2,1,1)
for p in paths:
    im = Image.open(paths.get_path(2,1,1))
    plt.imshow(im)
    plt.show()    
    

TypeError: 'path' object is not iterable

In [155]:
im = Image.open(paths.get_path(2,1,1))
plt.imshow(im)
plt.show()

In [121]:
paths = 'E:\zhouhan'

In [80]:
os.path.exists(paths)

True

In [23]:
lst=list('abcdefg')

In [25]:
dict(zip(range(1,len(lst)+1,1),lst))

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g'}

In [27]:
n_subdir=0
not n_subdir

True

In [90]:
i = 0
1 <= i <= 11

False

In [4]:
ss = 'nihk \tkkk '
ss.split(' ')

['nihk', '\tkkk', '']

In [5]:
a = ["foo", 2, "bar", 4, "far", 6]
group_adjacent = lambda x, k: zip(*([iter(x)] * k))
group_adjacent(a,2)

In [6]:
list(group_adjacent(a,2))

[('foo', 2), ('bar', 4), ('far', 6)]

In [11]:
list()

[<list_iterator at 0x1da537cf400>, <list_iterator at 0x1da537cf400>]

In [14]:
for i in zip([iter(a)] * 2):
    print(i)

(<list_iterator object at 0x000001DA537952B0>,)
(<list_iterator object at 0x000001DA537952B0>,)


In [16]:
list(zip(*[[1,2],[3,4]]))

[(1, 3), (2, 4)]

In [17]:
a = ["foo", 2, "bar", 4, "far", 6]
group_adjacent = lambda x, k: zip(*([iter(x),iter(x)]))
group_adjacent(a,2)

In [18]:
list(group_adjacent(a,2))

[('foo', 'foo'), (2, 2), ('bar', 'bar'), (4, 4), ('far', 'far'), (6, 6)]

In [19]:
dss='hello pyqt5'
s2=dss.join(['a','.','c'])
s2

'ahello pyqt5.hello pyqt5c'

In [21]:
dss='hello pyqt5'
s2=dss.join(['a','.','c'])
s2

'ahello pyqt5.hello pyqt5c'

In [25]:
dss.join(['a',''])

'ahello pyqt5'

In [26]:
'AjfksalD'.capitalize()

'Ajfksald'

In [28]:
list('AjfksalD').index('D')

7

In [32]:
list('AjfksalD').pop(-1)

'D'

In [35]:
list('AjfksalD').remove('c')

ValueError: list.remove(x): x not in list

In [39]:
lst = list('AjfksalD')
lst.insert(1,'b')
lst

['A', 'b', 'j', 'f', 'k', 's', 'a', 'l', 'D']

In [ ]:
list('AjfksalD').sort()

In [40]:
cmp(list('Ab'),list('B'))

NameError: name 'cmp' is not defined

In [ ]:
list('Bd').cmp()

In [41]:
list('Ab') > list('A')

True

In [42]:
str(list('ab'))

"['a', 'b']"

In [49]:
chr(111)

'o'

radiansdict.clear() 	删除字典内所有元素
radiansdict.copy() 	返回一个字典的浅复制
radiansdict.fromkeys() 	创建一个新字典，以序列seq中元素做字典的键，val为字典所有键对应的初始值
radiansdict.get(key, default=None) 	返回指定键的值，如果值不在字典中返回default值
radiansdict.has_key(key) 	如果键在字典dict里返回true，否则返回false
radiansdict.items() 	以列表返回可遍历的(键, 值) 元组数组
radiansdict.keys() 	以列表返回一个字典所有的键
radiansdict.setdefault(key, default=None) 	和get()类似, 但如果键不已经存在于字典中，将会添加键并将值设为default
radiansdict.update(dict2) 	把字典dict2的键/值对更新到dict里
radiansdict.values() 	以列表返回字典中的所有值

In [52]:
radiansdict = {1:'a'}
radiansdict

In [59]:
radiansdict.fromkeys([1,2,3],list('abc'))
radiansdict

NameError: name 'rdiansdict' is not defined

In [73]:
# radiansdict = {}
radiansdict.fromkeys([1,2])

{1: None, 2: None}

In [74]:
sdict = {}
sdict.fromkeys([1,2,3])
sdict.update(radiansdict)
sdict

{}

In [75]:
radiansdict = {1:'a',2:'b'}
radiansdict

{1: 'a', 2: 'b'}

In [78]:
radiansdict.update([(3,'c')])

In [79]:
radiansdict

{1: 'a', 2: 'b', 3: 'c'}

In [81]:
radiansdict.keys()

dict_keys([1, 2, 3])

In [82]:
import functools  # 需要先加载functools模块
def add(a, b):
    return a + b
plus3 = functools.partial(add, 3)
plus3(4)

7

In [85]:
class MyClass:
    name = "DefaultName"
    count = 0

    def __init__(self, name):
        self.name = name
        print('类的变量为%s\n对象的变量是%s\n' % ( MyClass.name, self.name))
    def setCount(self, count):
        self.count = count
        return None
    def getCount(self):
        return self.count

In [86]:
cls = MyClass('mine')

类的变量为DefaultName
对象的变量是mine



类的动态属性操作

In [87]:
class MyClass(object):
    def __init__(self):
        self._param = None
    def getParam(self):
        print( "get param: %s" % self._param)
        return self._param
    def setParam(self, value):
        print( "set param: %s" % self._param)
        self._param = value
    def delParam(self):
        print( "del param: %s" % self._param)
        del self._param
    
    param = property(getParam, setParam, delParam)

cls = MyClass()
cls.param = 10
print("current param : %s " % cls.param )
del cls.param

set param: None
get param: 10
current param : 10 
del param: 10


In [90]:
class MyClass(object):
    def __init__(self):
        self._param = None
    
    @property            # @property和@param.setter不同
    def param(self):
        print( "get param: %s" % self._param)
        return self._param
    
    @param.setter
    def param(self, value):  # 不同于setParam
        print( "set param: %s" % self._param)
        self._param = value
        
    @param.deleter
    def param(self):  # 不同于delParam
        print( "del param: %s" % self._param)
        del self._param

cls = MyClass()
cls.param = 10
print("current param : %s " % cls.param )
del cls.param

set param: None
get param: 10
current param : 10 
del param: 10
